In [98]:
import pandas as pd

In [99]:
url = "https://raw.githubusercontent.com/kodewilayah/permendagri-72-2019/main/dist/base.csv"
data = pd.read_csv(url)
data.columns = ["KODE", "WILAYAH"]

In [218]:
data.head(450)

,KODE,WILAYAH
0,11.01,KAB. ACEH SELATAN
1,11.01.01,Bakongan
2,11.01.01.2001,Keude Bakongan
3,11.01.01.2002,Ujong Mangki
4,11.01.01.2003,Ujong Padang
...,...,...
445,11.02.06.2025,Sabilussalam
446,11.02.06.2026,Lawe Desky Jaya
447,11.02.06.2027,Pintu Alas
448,11.02.07,Darul Hasanah


In [101]:
provinsi = {
    11: "Aceh",
    12: "Sumatera Utara",
    13: "Sumatera Barat",
    16: "Sumatera Selatan",
    14: "Riau",
    21: "Kepulauan Riau",
    15: "Jambi",
    17: "Bengkulu",
    18: "Lampung",
    19: "Bangka Belitung",
    31: "DKI Jakarta",
    36: "Banten",
    32: "Jawa Barat",
    33: "Jawa Tengah",
    34: "DI Yogyakarta",
    35: "Jawa Timur",
    51: "Bali",
    61: "Kalimantan Barat",
    62: "Kalimantan Tengah",
    63: "Kalimantan Selatan",
    64: "Kalimantan Timur",
    65: "Kalimantan Utara",
    52: "Nusa Tenggara Barat",
    53: "Nusa Tenggara Timur",
    71: "Sulawesi Utara",
    72: "Sulawesi Tengah",
    73: "Sulawesi Selatan",
    74: "Sulawesi Tenggara",
    75: "Gorontalo",
    76: "Sulawesi Barat",
    81: "Maluku",
    82: "Maluku Utara",
    91: "Papua",
    92: "Papua Barat"
}

In [104]:
data_count = data.count().iloc[0]
print(data_count)

91219


Change the csv data into json with format:


```
[
  {
    "kode_provinsi": "01"
    "kode_provinsi": "Aceh"
    "kabupaten": [
      "kode_kabupaten": "11.01"
      "nama_kabupaten": "KAB. ACEH SELATAN"
      "kecamatan": [
        "kode_kecamatan": "11.01.01"
        "nama_kecamatan": "Bakongan"
        "desa": [
          "kode_desa": "11.01.01.2001"
          "nama_desa": "Keude Bakongan"
        ]
      ]
    ]
  }.
  {
    ...
  },
  {
    ...
  },
  {
    ...
  }
]
```

In [290]:
clean_data = []

provinsi_array = []
kabupaten_array = []
kecamatan_array = []
desa_array = []

provinsi_dict = {}
kabupaten_dict = {}
kecamatan_dict = {}
desa_dict = {}

current_provinsi = None
current_kabupaten = None
current_kecamatan = None

last_kecamatan = None

iteration = 0

for i in range(data_count):
  kode = str(data["KODE"][i])
  wilayah = data["WILAYAH"][i]

  if (current_provinsi != None and current_provinsi != kode[:2]):
    clean_data.append(provinsi_dict.copy())
    kabupaten_array.clear()
    # iteration += 1
    # if iteration > 0:
    #   break

  current_provinsi = kode[:2]

  if len(kode) == 5:
    if current_kabupaten != None:
      kabupaten_array.append(kabupaten_dict.copy())
      kecamatan_array.clear()
    current_kabupaten = wilayah
    continue

  if len(kode) == 8:
    if current_kecamatan != None:
      kecamatan_array.append(kecamatan_dict.copy())
    current_kecamatan = wilayah
    desa_array.clear()
    continue

  if len(kode) > 8:
    kode_provinsi = kode[:2]
    nama_provinsi = provinsi[int(kode_provinsi)]
    kode_kabupaten = kode[:5]
    nama_kabupaten = current_kabupaten
    kode_kecamatan = kode[:8]
    nama_kecamatan = current_kecamatan
    kode_desa = kode
    nama_desa = wilayah

    desa_dict["kode_desa"] = kode_desa
    desa_dict["nama_desa"] = nama_desa
    desa_array.append(desa_dict.copy())

    kecamatan_dict["kode_kecamatan"] = kode_kecamatan
    kecamatan_dict["nama_kecamatan"] = nama_kecamatan
    kecamatan_dict["desa"] = desa_array.copy()

    kabupaten_dict["kode_kabupaten"] = kode_kabupaten
    kabupaten_dict["nama_kabupaten"] = nama_kabupaten
    kabupaten_dict["kecamatan"] = kecamatan_array.copy()

    provinsi_dict["kode_provinsi"] = kode_provinsi
    provinsi_dict["nama_provinsi"] = nama_provinsi
    provinsi_dict["kabupaten"] = kabupaten_array.copy()

    last_kecamatan = current_kecamatan
    # print("Prov: ", nama_provinsi, ", Kab: ", nama_kabupaten, ", Kec: ", nama_kecamatan, ", Desa: ", nama_desa)


# Add last provinsi, logic error, my mistake
clean_data.append(provinsi_dict.copy())
kabupaten_array.clear()

print(len(clean_data))

34


In [291]:
# Problem: the first element of the second array should be an element of the first array
print(len(clean_data[0]["kabupaten"][0]["kecamatan"]), " - ", clean_data[0]["kabupaten"][0]["kecamatan"])
print(len(clean_data[0]["kabupaten"][1]["kecamatan"]), " - ", clean_data[0]["kabupaten"][1]["kecamatan"])
print(len(clean_data[0]["kabupaten"][2]["kecamatan"]), " - ", clean_data[0]["kabupaten"][2]["kecamatan"])
print(len(clean_data[0]["kabupaten"][3]["kecamatan"]), " - ", clean_data[0]["kabupaten"][3]["kecamatan"])

17  -  [{'kode_kecamatan': '11.01.01', 'nama_kecamatan': 'Bakongan', 'desa': [{'kode_desa': '11.01.01.2001', 'nama_desa': 'Keude Bakongan'}, {'kode_desa': '11.01.01.2002', 'nama_desa': 'Ujong Mangki'}, {'kode_desa': '11.01.01.2003', 'nama_desa': 'Ujong Padang'}, {'kode_desa': '11.01.01.2004', 'nama_desa': 'Gampong Drien'}, {'kode_desa': '11.01.01.2015', 'nama_desa': 'Darul Ikhsan'}, {'kode_desa': '11.01.01.2016', 'nama_desa': 'Padang Beurahan'}, {'kode_desa': '11.01.01.2017', 'nama_desa': 'Gampong Baro'}]}, {'kode_kecamatan': '11.01.02', 'nama_kecamatan': 'Kluet Utara', 'desa': [{'kode_desa': '11.01.02.2001', 'nama_desa': 'Fajar Harapan'}, {'kode_desa': '11.01.02.2002', 'nama_desa': 'Krueng Batee'}, {'kode_desa': '11.01.02.2003', 'nama_desa': 'Pasi Kuala Asahan'}, {'kode_desa': '11.01.02.2004', 'nama_desa': 'Gunung Pulo'}, {'kode_desa': '11.01.02.2005', 'nama_desa': 'Pulo Ie I'}, {'kode_desa': '11.01.02.2006', 'nama_desa': 'Jambo Manyang'}, {'kode_desa': '11.01.02.2007', 'nama_desa': '

In [292]:
# Solving the problem
edited_clean_data = clean_data.copy()
for i in range(len(clean_data)):
  for j in range(len(clean_data[i]["kabupaten"]) - 1):
    data_to_move = clean_data[i]["kabupaten"][j + 1]["kecamatan"][0].copy()
    edited_clean_data[i]["kabupaten"][j]["kecamatan"].append(data_to_move)
    del edited_clean_data[i]["kabupaten"][j + 1]["kecamatan"][0]

print(len(edited_clean_data[0]["kabupaten"][0]["kecamatan"]), " - ", edited_clean_data[0]["kabupaten"][0]["kecamatan"])
print(len(edited_clean_data[0]["kabupaten"][1]["kecamatan"]), " - ", edited_clean_data[0]["kabupaten"][1]["kecamatan"])
print(len(edited_clean_data[0]["kabupaten"][2]["kecamatan"]), " - ", edited_clean_data[0]["kabupaten"][2]["kecamatan"])
print(len(edited_clean_data[0]["kabupaten"][3]["kecamatan"]), " - ", edited_clean_data[0]["kabupaten"][3]["kecamatan"])


18  -  [{'kode_kecamatan': '11.01.01', 'nama_kecamatan': 'Bakongan', 'desa': [{'kode_desa': '11.01.01.2001', 'nama_desa': 'Keude Bakongan'}, {'kode_desa': '11.01.01.2002', 'nama_desa': 'Ujong Mangki'}, {'kode_desa': '11.01.01.2003', 'nama_desa': 'Ujong Padang'}, {'kode_desa': '11.01.01.2004', 'nama_desa': 'Gampong Drien'}, {'kode_desa': '11.01.01.2015', 'nama_desa': 'Darul Ikhsan'}, {'kode_desa': '11.01.01.2016', 'nama_desa': 'Padang Beurahan'}, {'kode_desa': '11.01.01.2017', 'nama_desa': 'Gampong Baro'}]}, {'kode_kecamatan': '11.01.02', 'nama_kecamatan': 'Kluet Utara', 'desa': [{'kode_desa': '11.01.02.2001', 'nama_desa': 'Fajar Harapan'}, {'kode_desa': '11.01.02.2002', 'nama_desa': 'Krueng Batee'}, {'kode_desa': '11.01.02.2003', 'nama_desa': 'Pasi Kuala Asahan'}, {'kode_desa': '11.01.02.2004', 'nama_desa': 'Gunung Pulo'}, {'kode_desa': '11.01.02.2005', 'nama_desa': 'Pulo Ie I'}, {'kode_desa': '11.01.02.2006', 'nama_desa': 'Jambo Manyang'}, {'kode_desa': '11.01.02.2007', 'nama_desa': '

In [293]:
# Problem: the first element of the second array should be an element of the first array
for k in range(len(edited_clean_data)):
  print(len(edited_clean_data[k]["kabupaten"]), " - ", edited_clean_data[k]["kabupaten"])
  for l in range(len(edited_clean_data[k]["kabupaten"])):
    print(edited_clean_data[k]["kabupaten"][l]["nama_kabupaten"])
  if k > 2:
    break


22  -  [{'kode_kabupaten': '11.01', 'nama_kabupaten': 'KAB. ACEH SELATAN', 'kecamatan': [{'kode_kecamatan': '11.01.01', 'nama_kecamatan': 'Bakongan', 'desa': [{'kode_desa': '11.01.01.2001', 'nama_desa': 'Keude Bakongan'}, {'kode_desa': '11.01.01.2002', 'nama_desa': 'Ujong Mangki'}, {'kode_desa': '11.01.01.2003', 'nama_desa': 'Ujong Padang'}, {'kode_desa': '11.01.01.2004', 'nama_desa': 'Gampong Drien'}, {'kode_desa': '11.01.01.2015', 'nama_desa': 'Darul Ikhsan'}, {'kode_desa': '11.01.01.2016', 'nama_desa': 'Padang Beurahan'}, {'kode_desa': '11.01.01.2017', 'nama_desa': 'Gampong Baro'}]}, {'kode_kecamatan': '11.01.02', 'nama_kecamatan': 'Kluet Utara', 'desa': [{'kode_desa': '11.01.02.2001', 'nama_desa': 'Fajar Harapan'}, {'kode_desa': '11.01.02.2002', 'nama_desa': 'Krueng Batee'}, {'kode_desa': '11.01.02.2003', 'nama_desa': 'Pasi Kuala Asahan'}, {'kode_desa': '11.01.02.2004', 'nama_desa': 'Gunung Pulo'}, {'kode_desa': '11.01.02.2005', 'nama_desa': 'Pulo Ie I'}, {'kode_desa': '11.01.02.20

In [294]:
# Solving the problem
for k in range(len(edited_clean_data) - 1):
  data_to_move = clean_data[k + 1]["kabupaten"][0].copy()
  edited_clean_data[k]["kabupaten"].append(data_to_move)
  del edited_clean_data[k + 1]["kabupaten"][0]

In [ ]:
for k in range(len(edited_clean_data)):
  print(len(edited_clean_data[k]["kabupaten"]), " - ", edited_clean_data[k]["kabupaten"])
  for l in range(len(edited_clean_data[k]["kabupaten"])):
    print(edited_clean_data[k]["kabupaten"][l]["nama_kabupaten"])
  if k > 2:
    break

In [296]:
import json

# convert into json
final = json.dumps(clean_data, indent=4)
with open("kode_wilayah.json", "w") as outfile:
    outfile.write(final)

In [297]:
data.head(20)

,KODE,WILAYAH
0,11.01,KAB. ACEH SELATAN
1,11.01.01,Bakongan
2,11.01.01.2001,Keude Bakongan
3,11.01.01.2002,Ujong Mangki
4,11.01.01.2003,Ujong Padang
5,11.01.01.2004,Gampong Drien
6,11.01.01.2015,Darul Ikhsan
7,11.01.01.2016,Padang Beurahan
8,11.01.01.2017,Gampong Baro
9,11.01.02,Kluet Utara
